In [6]:
library(dplyr, warn.conflicts = FALSE)
library(moments)
library(ggplot2)
library(repr)
source('multiplot.r')
options(repr.plot.width=10, repr.plot.height=2) 

In [7]:
housing_data <- data.frame(read.table(file='../data/housing.data', header=FALSE, sep=""))
colnames(housing_data) <- c('CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV')
housing_data$MEDV <- NULL
housing_data$CHAS = factor(housing_data$CHAS)
housing_features = Filter(is.numeric, housing_data)

ERROR: Error in eval(expr, envir, enclos): could not find function "housing_data.describe"


# Outlier Analysis (Extra Credit)

### Use Tukey's method to identify outliers for each feature

### Identify each instance that is an outlier for more than one feature

### Assess what percentage of the total data are outliers for:
    - one feature
    - two features
    - other

### Come up with a plan for handling outliers